In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import logging
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os


spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

In [16]:
df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/simbox_21_10_2021_to_07_11_2021.csv")
df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("225") , col("msisdn")))                                                                                     
df_fraudeur_FRA.show(5)                                                                                   

+--------+-------------+-----+---------+-----------+
| date_id|       msisdn|outil|operateur|destination|
+--------+-------------+-----+---------+-----------+
|20211021|2250700022953|   FR|   ORANGE|     ON NET|
|20211021|2250700022954|   FR|   ORANGE|     ON NET|
|20211021|2250700023060|   FR|   ORANGE|     ON NET|
|20211021|2250700057473|   FR|   ORANGE|     ON NET|
|20211021|2250700180629|   FR|   ORANGE|     ON NET|
+--------+-------------+-----+---------+-----------+
only showing top 5 rows



In [17]:
df_fraudeur_FRA.filter(col("date_id") == "20211101").select("msisdn").distinct().count()

108

In [26]:
df_fraudeur_FRA.count()

4753

In [27]:
df_fraudeur_FRA.dropDuplicates(["msisdn", "date_id"]).count()

4609

In [17]:
def check_fraudeur_date(date) :
    from datetime import datetime

    path = "/Users/youssouf/Downloads/predition_" + date + ".csv"
    df = spark.read.option("header", "true").option("delimiter",";").csv(path)
      
    df_fraudeur = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/simbox_21_10_2021_to_07_11_2021.csv") \
                        .withColumn("msisdn", concat(lit("225") , col("msisdn"))) \
                        .filter(col("date_id") == date)
        
    compa = df.join(df_fraudeur , ["msisdn"], "inner")
    
    a = df.join(df_fraudeur, ["msisdn"], "inner").count() 
    res = {"NOMBRE DE DETECTION" : df.select("msisdn").distinct().count()  , 
           "NOMBRE DE FRA" : df_fraudeur.select("msisdn").distinct().count() ,
           "NOMBRE COMMUN" : compa.select("msisdn").distinct().count() 
           #"NUMERO COMMUN" : compa.select("msisdn").show(a) ,
           #"NOMBRE FRAU/NON-FRAU": "" #df.groupBy("prediction").count().show() 
          }

    return res

In [18]:
check_fraudeur_date("20211021")

{'NOMBRE DE DETECTION': 731813, 'NOMBRE DE FRA': 338, 'NOMBRE COMMUN': 138}

In [19]:
check_fraudeur_date("20211022")

{'NOMBRE DE DETECTION': 683733, 'NOMBRE DE FRA': 221, 'NOMBRE COMMUN': 91}

In [20]:
check_fraudeur_date("20211023")

{'NOMBRE DE DETECTION': 629745, 'NOMBRE DE FRA': 210, 'NOMBRE COMMUN': 120}

In [23]:
df_nov = spark.read.option("header","true").option("delimiter",";").csv("/Users/youssouf/Downloads/df_trafic_simbox_nov_ok3.csv")
df_nov.show(5)                                                   
df_nov.count()                                                  
                                                   
                                                   

+--------+----------+-----+---------+-----------+----------+
| date_id|    msisdn|outil|operateur|destination|DATE_APPEL|
+--------+----------+-----+---------+-----------+----------+
|20211106|0778959263|   FR|   ORANGE|     ON NET|  20211106|
|20211107|0704582743|   FR|   ORANGE|     ON NET|  20211107|
|20211107|0704582645|   FR|   ORANGE|     ON NET|  20211107|
|20211105|0799760619|   FR|   ORANGE|     ON NET|  20211105|
|20211105|0700055419|   FR|   ORANGE|     ON NET|  20211105|
+--------+----------+-----+---------+-----------+----------+
only showing top 5 rows



10090

In [24]:
df_nov_ = df_nov.dropDuplicates(["msisdn", "date_id"])

In [25]:
df_nov_.count()

727

In [45]:
df_nov_1 = spark.read.option("header","true").option("delimiter",";").csv("/Users/youssouf/Downloads/df_trafic_simbox_nov_ok2.csv")
df_nov_1.show(10)




+--------+----------+-----+---------+-----------+----------+
| date_id|    msisdn|outil|operateur|destination|DATE_APPEL|
+--------+----------+-----+---------+-----------+----------+
|20211102|0767821465|   FR|   ORANGE|     ON NET|  20211102|
|20211102|0704514312|   FR|   ORANGE|     ON NET|  20211102|
|20211101|0799941475|   FR|   ORANGE|     ON NET|  20211101|
|20211101|0704241555|   FR|   ORANGE|     ON NET|  20211101|
|20211101|0778571903|   FR|   ORANGE|     ON NET|  20211101|
|20211102|0704422848|   FR|   ORANGE|     ON NET|  20211102|
|20211103|0798967742|   FR|   ORANGE|     ON NET|  20211103|
|20211102|0704243898|   FR|   ORANGE|     ON NET|  20211102|
|20211104|0768466296|   FR|   ORANGE|     ON NET|  20211104|
|20211104|0799179101|   FR|   ORANGE|     ON NET|  20211104|
+--------+----------+-----+---------+-----------+----------+
only showing top 10 rows



In [46]:
df_nov_2 = spark.read.option("header","true").option("delimiter",";").csv("/Users/youssouf/Downloads/df_trafic_simbox_nov_ok1.csv")
df_nov_2.show(10)


+--------+----------+-----+---------+-----------+----------+
| date_id|    msisdn|outil|operateur|destination|DATE_APPEL|
+--------+----------+-----+---------+-----------+----------+
|20211027|0704182269|   FR|   ORANGE|     ON NET|  20211027|
|20211028|0704184034|   FR|   ORANGE|     ON NET|  20211028|
|20211028|0704182357|   FR|   ORANGE|     ON NET|  20211028|
|20211027|0778233184|   FR|   ORANGE|     ON NET|  20211027|
|20211027|0799047677|   FR|   ORANGE|     ON NET|  20211027|
|20211027|0799873892|   FR|   ORANGE|     ON NET|  20211027|
|20211027|0799873863|   FR|   ORANGE|     ON NET|  20211027|
|20211026|0704552025|   FR|   ORANGE|     ON NET|  20211026|
|20211026|0704044992|   FR|   ORANGE|     ON NET|  20211026|
|20211028|0749740303|   FR|   ORANGE|     ON NET|  20211028|
+--------+----------+-----+---------+-----------+----------+
only showing top 10 rows



In [47]:
df_nov_3 = spark.read.option("header","true").option("delimiter",";").csv("/Users/youssouf/Downloads/df_trafic_simbox_nov_ok.csv")
df_nov_3.show(10)



+--------+----------+-----+---------+-----------+----------+
| date_id|    msisdn|outil|operateur|destination|DATE_APPEL|
+--------+----------+-----+---------+-----------+----------+
|20211023|0704307953|   FR|   ORANGE|     ON NET|  20211023|
|20211022|0787517713|   FR|   ORANGE|     ON NET|  20211022|
|20211021|0704425565|   FR|   ORANGE|     ON NET|  20211021|
|20211021|0704320738|   FR|   ORANGE|     ON NET|  20211021|
|20211025|0749639772|   FR|   ORANGE|     ON NET|  20211025|
|20211025|0758415347|   FR|   ORANGE|     ON NET|  20211025|
|20211021|0769243852|   FR|   ORANGE|     ON NET|  20211021|
|20211021|0700502573|   FR|   ORANGE|     ON NET|  20211021|
|20211021|0788278884|   FR|   ORANGE|     ON NET|  20211021|
|20211021|0799795741|   FR|   ORANGE|     ON NET|  20211021|
+--------+----------+-----+---------+-----------+----------+
only showing top 10 rows



In [38]:
df_fr_trafic = df_nov_3.union(df_nov_2).union(df_nov_1).union(df_nov)

In [39]:
df_fr_trafic.write.mode("overwrite").format("parquet").save("/Users/youssouf/Documents/simboxv2/fraudeur_20211021_20211107_only_trafic.parquet")




In [52]:
def check_fraudeur_date2(date) :
    from datetime import datetime

    path = "/Users/youssouf/Downloads/predition_" + date + ".csv"
    df = spark.read.option("header", "true").option("delimiter",";").csv(path)
         
    path_fr = "/Users/youssouf/Documents/simboxv2/fraudeur_20211021_20211107_only_trafic.parquet"
    df_fraudeur = spark.read.parquet(path_fr) \
                        .withColumn("msisdn", concat(lit("225") , col("msisdn"))) \
                        .dropDuplicates(["msisdn", "date_id"]) \
                        .filter(col("date_id") == date)
    
    compa = df.join(df_fraudeur , ["msisdn"], "inner")
    
    a = df.join(df_fraudeur, ["msisdn"], "inner").count() 
    res = {"NOMBRE DE DETECTION" : df.select("msisdn").distinct().count()  , 
           "NOMBRE DE FRA" : df_fraudeur.select("msisdn").distinct().count() ,
           "NOMBRE COMMUN" : compa.select("msisdn").distinct().count() , 
           "DATE" : date
          }

    return res

In [53]:
check_fraudeur_date2("20211021")

{'NOMBRE DE DETECTION': 731813,
 'NOMBRE DE FRA': 284,
 'NOMBRE COMMUN': 138,
 'DATE': '20211021'}

In [54]:
check_fraudeur_date2("20211022")

{'NOMBRE DE DETECTION': 683733,
 'NOMBRE DE FRA': 218,
 'NOMBRE COMMUN': 91,
 'DATE': '20211022'}

In [55]:
check_fraudeur_date2("20211023")

{'NOMBRE DE DETECTION': 629745,
 'NOMBRE DE FRA': 209,
 'NOMBRE COMMUN': 120,
 'DATE': '20211023'}

In [56]:
120/209

0.5741626794258373

In [57]:
138/284

0.4859154929577465

In [58]:
91/218

0.41743119266055045

In [14]:
path_fr = "/Users/youssouf/Documents/simboxv2/fraudeur_20211021_20211107_only_trafic.parquet"
df_fraudeur = spark.read.parquet(path_fr) \
                        .withColumn("msisdn", concat(lit("225") , col("msisdn"))) \
                        .dropDuplicates(["msisdn", "date_id"]) \
                        .filter(col("date_id") == "20211101")
df_fraudeur.show(5)

+--------+-------------+-----+---------+-----------+----------+
| date_id|       msisdn|outil|operateur|destination|DATE_APPEL|
+--------+-------------+-----+---------+-----------+----------+
|20211101|2250799158468|   FR|   ORANGE|     ON NET|  20211101|
|20211101|2250799636673|   FR|   ORANGE|     ON NET|  20211101|
|20211101|2250777241106|   FR|   ORANGE|     ON NET|  20211101|
|20211101|2250757015477|   FR|   ORANGE|     ON NET|  20211101|
|20211101|2250777162304|   FR|   ORANGE|     ON NET|  20211101|
+--------+-------------+-----+---------+-----------+----------+
only showing top 5 rows



In [15]:
df_fraudeur.count()

108

In [1]:
import pandas as pd

In [2]:
vv = pd.read_csv("/Users/youssouf/Downloads/A_regarder.csv")
vv.head()

,x
0,2720329871
1,2720334484
2,2720227529
3,2720224271
4,2720227704


In [ ]:
"2720329871",
"2720334484",
"2720227529",
"2720224271",
"2720227704",
"2720221420",
"2720223388",
"2720218889",
"2720221233",
"2720214801",
"2720210012",
"2722422480",
"2722420407",
"2722447169",
"2722443131",
"2722442998",
"2722419000",
"2722416367",
"2721243260",
"2721248925"

In [11]:
vv.x.apply(str).to_list()

['2720329871',
 '2720334484',
 '2720227529',
 '2720224271',
 '2720227704',
 '2720221420',
 '2720223388',
 '2720218889',
 '2720221233',
 '2720214801',
 '2720210012',
 '2722422480',
 '2722420407',
 '2722447169',
 '2722443131',
 '2722442998',
 '2722419000',
 '2722416367',
 '2721243260',
 '2721248925']

In [8]:
vv.x.apply(lambda x: "0" + str(x) if (len(str(x)) == 12) )

SyntaxError: invalid syntax (<ipython-input-8-4085a73d9fc7>, line 1)

In [9]:
def f(x) :
    if len(str(x)) == 11 :
        return "0" + str(x)

In [10]:
vv.x.apply(f)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: x, dtype: object